In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# before this step, connect with google drive
%cd /content/drive/MyDrive/cs679_project_compare_40/code/model_facebook

/content/drive/MyDrive/cs679_project_compare_40/code/model_facebook


Metadata: [filename, speaker embedding, spectrogram]

Modify Speaker Embedding

In [3]:
import glob
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import torch
import librosa
import torch.nn as nn
# For example, using the "wav2vec2-large-xlsr-53" model. Adjust the model name based on your needs.
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:365: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
if not torch.cuda.is_available():
    print("CUDA is not available. Using CPU instead.")
    device = torch.device("cpu")
else:
    print("CUDA is available. Using GPU.")
    device = torch.device("cuda")

# Initialize the processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base").to(device)

def extract_embedding(audio_path):
    waveform, sampling_rate = librosa.load(audio_path, sr=16000)

    inputs = processor(waveform, sampling_rate=16000, return_tensors="pt", padding=True).to(device)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state

    # Define the linear layer (assuming the size of the last hidden layer of the model is 768)
    linear = nn.Linear(768, 256, bias = True).to(device)
    projected_embeddings = linear(embeddings)
    embeddings = linear(embeddings)  
    return embeddings[:,-1,:].detach().squeeze().cpu().numpy()

CUDA is available. Using GPU.


/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:365: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
embed = extract_embedding('/content/drive/MyDrive/cs679_project_compare_40/wavs/p225/p225_001.wav')
embed.shape

(256,)

In [27]:
from tqdm import tqdm
def embedding_each_wav(folder_path):
    all_embeddings = []
    for audio_path in tqdm(glob.glob(folder_path + '*.wav')):
        embedding = extract_embedding(audio_path)
        all_embeddings.append(embedding)
    average_embedding = np.mean(all_embeddings, axis = 0)
    return average_embedding

Modify Spectrogram

In [30]:
import pandas as pd
import numpy as np
import pickle

In [31]:
df = pd.read_pickle('original/metadata.pkl')
metadata_0 = [df[i][0] for i in range(len(df))]
#wav_names = [name+'x'+name+'.wav' for name in metadata_0]
folder_names = ['wavs/'+name+'/' for name in metadata_0]
folder_names

['wavs/p225/',
 'wavs/p226/',
 'wavs/p227/',
 'wavs/p228/',
 'wavs/p229/',
 'wavs/p230/',
 'wavs/p231/',
 'wavs/p232/',
 'wavs/p233/',
 'wavs/p234/',
 'wavs/p236/',
 'wavs/p237/',
 'wavs/p238/',
 'wavs/p239/',
 'wavs/p240/',
 'wavs/p241/',
 'wavs/p243/',
 'wavs/p244/',
 'wavs/p245/',
 'wavs/p246/',
 'wavs/p247/',
 'wavs/p248/',
 'wavs/p249/',
 'wavs/p250/',
 'wavs/p251/',
 'wavs/p252/',
 'wavs/p253/',
 'wavs/p254/',
 'wavs/p255/',
 'wavs/p256/',
 'wavs/p257/',
 'wavs/p258/',
 'wavs/p259/',
 'wavs/p260/',
 'wavs/p261/',
 'wavs/p262/',
 'wavs/p263/',
 'wavs/p264/',
 'wavs/p265/',
 'wavs/p266/']

In [34]:
for i in range(len(df)):
    df[i][1] = embedding_each_wav(folder_names[i])
    with open('temp.pkl', 'wb') as file:
        pickle.dump(df, file)


100%|██████████| 422/422 [00:21<00:00, 19.26it/s]


In [35]:
with open('metadata_facebook.pkl', 'wb') as file:
    pickle.dump(df, file)

In [36]:
df = pd.read_pickle('metadata_facebook.pkl')
df[0][1].shape

(256,)

In [37]:
df[0][1]

array([ 7.0925122e-03, -2.7549120e-02, -2.1464389e-03, -2.9718208e-03,
       -2.1087632e-02, -3.1837465e-03,  2.3327734e-02,  1.2455463e-02,
        7.6666311e-03, -5.0971014e-03, -1.9102631e-02,  3.1587963e-03,
        9.3108667e-03, -9.6659716e-03,  1.7902719e-02, -5.0601400e-03,
       -1.0414773e-02,  2.3798555e-02,  1.4051576e-03,  1.0644793e-02,
        1.3062449e-03, -3.8082726e-02,  2.1130366e-03, -8.8709919e-03,
       -2.7267959e-02, -1.5696393e-03, -1.6965659e-03, -4.8795734e-03,
        1.0077662e-02, -2.5315287e-02, -5.9927162e-03, -2.8125205e-04,
        1.4406303e-03,  8.2016680e-03,  7.0929495e-03, -1.4930954e-02,
        1.9972827e-02,  1.0674876e-02, -2.7422223e-03, -1.3453321e-02,
       -2.8382901e-03, -1.7240418e-04, -6.8904073e-03, -1.7622437e-02,
        7.0353034e-03,  2.3210622e-02,  5.7909340e-03,  3.8445005e-03,
        7.8888768e-03, -1.9402074e-02,  4.9055222e-02, -1.6271516e-03,
       -1.4684755e-02,  8.8991848e-04, -2.0855153e-02,  3.2005433e-03,
      